In [52]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import KeyedVectors
import numpy as np
from typing import List

In [53]:
word_vectors = KeyedVectors.load_word2vec_format("D:\\nlp\\vectors\\news.lowercased.tokenized.word2vec.300d", binary=False)


In [54]:
result = word_vectors.most_similar(positive=['україна', 'саакашвілі'], negative=['порошенко'])
print(result)

[('грузія', 0.5679004192352295), ('**україна', 0.5457552671432495), ('росія', 0.5360526442527771), ('країна', 0.5280444622039795), ('абхазія', 0.5090128779411316), ('європа', 0.5020069479942322), ('**_україна', 0.4978522062301636), ('америка', 0.4906144142150879), ('туреччина', 0.4863477349281311), ('греція', 0.47572413086891174)]


In [55]:
print(word_vectors.most_similar(positive=['тимошенко', 'чоловік'], negative=['жінка']))
print('белиберда123132' in word_vectors)
# print(word_vectors.word_vec('белиберда123132', ))

[('луценко', 0.4683634042739868), ('тимошенко**', 0.45959794521331787), ('володимирівни', 0.4512634873390198), ('луценка', 0.44807755947113037), ("екс-прем'єра", 0.43939143419265747), ('тимошенко_', 0.4320363700389862), ('турчинов', 0.42672640085220337), ('яценюк', 0.42651936411857605), ('мороз', 0.4248889982700348), ('тимо\xadшенко', 0.4224357604980469)]
False


In [56]:
single_vec = word_vectors.word_vec('тест')
print(single_vec.shape)

(300,)


In [57]:
import re
re_tokens = re.compile("([\w][\w]*'?\w?)")

def text2tokens(text: str)->List[str]:
    return re_tokens.findall(text)

s = "Ваша відповідь:  Повідомляємо, що відповідно до п. 5.7 Державного стандарту України 4123-2006 «Пристрій примусового зниження швидкості дорожньо-транспортної техніки на вулицях і дорогах. Загальні технічні вимоги» заборонено встановлювати пристрої примусового зниження швидкості руху транспортних засобів на магістральних вулицях, а просп. Григоренка має категорію магістральної вулиці районного значення."
print(text2tokens(s))

['Ваша', 'відповідь', 'Повідомляємо', 'що', 'відповідно', 'до', 'п', '5', '7', 'Державного', 'стандарту', 'України', '4123', '2006', 'Пристрій', 'примусового', 'зниження', 'швидкості', 'дорожньо', 'транспортної', 'техніки', 'на', 'вулицях', 'і', 'дорогах', 'Загальні', 'технічні', 'вимоги', 'заборонено', 'встановлювати', 'пристрої', 'примусового', 'зниження', 'швидкості', 'руху', 'транспортних', 'засобів', 'на', 'магістральних', 'вулицях', 'а', 'просп', 'Григоренка', 'має', 'категорію', 'магістральної', 'вулиці', 'районного', 'значення']


In [66]:
from uk_stopwords import STOP_WORDS as STOP_WORDS_UK
from ru_stopwords import STOP_WORDS as STOP_WORDS_RU

STOP_WORDS = STOP_WORDS_UK.union(STOP_WORDS_RU)

def text2vec(text: str):
    res = np.zeros(single_vec.shape)
    words = text2tokens(text.lower())
    for w in words:
        if w in STOP_WORDS:
            continue
        if w in word_vectors:
            v = word_vectors.word_vec(w)
            res += v
    res /= max(np.linalg.norm(res), 1e-6)
    return res

print(text2vec(s))

[-1.06386394e-01  2.16623768e-02 -1.49693291e-02  8.28187498e-02
  9.41666734e-02  2.76507260e-02  6.26679588e-02 -9.93558682e-02
 -6.32134774e-02 -8.21703079e-02 -6.91073556e-02 -1.48737746e-01
  4.92090944e-02  2.21420203e-02 -2.62377465e-02 -4.51006791e-03
  2.23751767e-02 -3.74677797e-02 -3.76029777e-02 -8.37533050e-02
  1.06504312e-02  4.38264432e-02 -1.38292100e-02 -5.44874369e-02
 -1.54762311e-01 -4.02460176e-02 -2.52238424e-02  1.46450416e-02
 -2.40280742e-02  4.37515763e-02 -3.66224077e-03  1.02151102e-01
  3.17945475e-02  1.36391636e-02  1.47711384e-03 -9.52284098e-02
  1.32992767e-02 -1.93077648e-02 -5.44707315e-02  2.68498569e-02
  4.08563771e-02 -1.60910555e-03  5.75545601e-02  6.67588320e-02
 -1.05698077e-01  8.71921018e-02  8.28533312e-02  1.86119576e-02
  2.92577815e-02  2.46378261e-02  5.21778023e-03 -1.91028280e-02
  4.22079201e-02  1.02510298e-01 -1.40543200e-01  2.10621166e-02
  5.21049740e-02 -2.12989894e-02 -7.21386182e-03 -2.10125661e-02
  1.31671772e-02 -2.57162

In [67]:
import os

def read_category(fname: str):
    res = []
    with open(fname, 'rt', encoding='utf-8') as f:
        buff = []
        prev_was_empty_line = False
        def flush_buff():
            if buff:
                res.append('\n'.join(buff))
            buff.clear()
            prev_was_empty_line = False
            
        for line in f:
            l = line.strip()
            if not l and prev_was_empty_line:
                flush_buff()
            else:
                buff.append(l)
            prev_was_empty_line = not l
        flush_buff()
    return res

def load_texts():
    res = {}
    root_dir = 'data/1551'
    filenames = os.listdir(path=root_dir)
    for fn in filenames:
        messages = read_category(os.path.join(root_dir, fn))
        res[fn] = messages
    return res

all_messages = load_texts()

In [68]:
print(len(all_messages))
print([(cat, len(all_messages[cat])) for cat in all_messages])

278
[('--Корчування-пнів.txt', 1), ('Інші-питання--пропозиції-щодо-роботи-міської-електрички.txt', 1), ('Інші-питання-утримання-малих-архітектурних-форм-та-прилеглої-до-них-території.txt', 1), ('Інші-Подяки.txt', 9), ('Інші-порушення-громадського-порядку-і-громадської-безпеки.txt', 2), ('Інші-провайдери-кабельного-телебачення.txt', 1), ('Інші-технічні-недоліки-стану-ліфту.txt', 5), ('Аварійний--травмонебезпечний-стан-утримання-об-єктів-благоустрою.txt', 12), ('Аварійних-служб--Ліфтові-служби---Київенерго---Київвводоканал--тощо-.txt', 1), ('Асфальтування-тролейбусних--зупинок.txt', 1), ('Бажаючі-отримати--Картки-киянина--КК--.txt', 2), ('Безпритульні-люди-у-будинку.txt', 1), ('Благоустрій-поховань-та-їх-санітарний-стан.txt', 1), ('Будівництво--дооблаштування-спортивних-майданчиків.txt', 2), ('Будівництво-АЗС.txt', 5), ('Будівництво-в-нічний-час.txt', 1), ('Будівництво-дооблаштування-дитячого-майданчику.txt', 19), ('Будівництво-каналізаційних-систем-у-приватному-секторі.txt', 3), ('Будів

In [69]:
for cat_name in all_messages:
    vec = text2vec('\n'.join(all_messages[cat_name]))
    #print(vec[:5])
    #print(all_messages[cat_name][0])
    print("Category:", cat_name)
    print(word_vectors.similar_by_vector(vec))

Category: --Корчування-пнів.txt
[('дерево', 0.540582537651062), ('полотно', 0.5226589441299438), ('мощення', 0.521578311920166), ('перехрестя', 0.5148742198944092), ('кріпосний', 0.49514222145080566), ('гілля', 0.48391440510749817), ('мозаїчне', 0.4792362153530121), ('бронзове', 0.4774654507637024), ('огородження', 0.47685903310775757), ('плиткою', 0.475751668214798)]
Category: Інші-питання--пропозиції-щодо-роботи-міської-електрички.txt
[('городня', 0.685028076171875), ('видубичі', 0.6772301197052002), ('київ-петрівка', 0.6607034206390381), ('борщагівка-технічна', 0.654832661151886), ('святошин', 0.6386718153953552), ('коростень', 0.6357401013374329), ('станції', 0.6313599348068237), ('красноград', 0.6312097907066345), ('київ-', 0.6299517750740051), ('козятин', 0.6262016296386719)]
Category: Інші-питання-утримання-малих-архітектурних-форм-та-прилеглої-до-них-території.txt
[('вид', 0.866919755935669), ('різновид', 0.6898993849754333), ('тип', 0.6595279574394226), ('мікроорганізм', 0.560

[('працює', 0.637418270111084), ('функціонує', 0.5514050722122192), ('працюватиме', 0.5296276807785034), ('горить', 0.5076162815093994), ('димить', 0.5036722421646118), ('вмикається', 0.49738064408302307), ('пошкоджено', 0.48981642723083496), ('працювала', 0.4877552092075348), ('чергує', 0.48267972469329834), ('працювало', 0.47292283177375793)]
Category: Відсутній-тиск-ХВП-до---го-поверху-включно---Київводоканал---.txt
[('вода', 0.6258628368377686), ('води', 0.5417181253433228), ('рідина', 0.523333728313446), ('водичка', 0.5170520544052124), ('волога', 0.5097354650497437), ('каналізація', 0.5044054985046387), ('електрика', 0.5008214712142944), ('ванна', 0.4932946562767029), ('електроенергія', 0.4856790006160736), ('кондиціонера', 0.4794403910636902)]
Category: Відсутній-тиск-ХВП-з----го-поверху-включно---Київенерго--.txt
[('води', 0.5965508818626404), ('опалення', 0.5897656083106995), ('опалювання', 0.58754962682724), ('каналізації', 0.5656343102455139), ('теплоносія', 0.54851919412612

[('словно', 0.7645977735519409), ('надпись', 0.7521784901618958), ('стены', 0.7470988035202026), ('вашей', 0.7468156814575195), ('моей', 0.7411183714866638), ('услуги', 0.7398616075515747), ('девушки', 0.7392847537994385), ('комнаты', 0.739173412322998), ('ткани', 0.7384944558143616), ('здание', 0.7381865978240967)]
Category: Впровадження-нових-тролейбусних-зупинок.txt
[('трех', 0.8713899850845337), ('объекта', 0.8682088851928711), ('отдельных', 0.8652638792991638), ('непосредственно', 0.8634710311889648), ('преодоление', 0.8627643585205078), ('официально', 0.8606644868850708), ('результатов', 0.8592273592948914), ('взаимодействия', 0.8592046499252319), ('препараты', 0.8578881025314331), ('ранее', 0.8577666282653809)]
Category: Встановлення--розміщення--і-демонтаж-біотуалетів.txt
[('ковролін', 0.5348429679870605), ('картоном', 0.5190505981445312), ('мощення', 0.5167351961135864), ('ангар', 0.5070443153381348), ('засклення', 0.5069035887718201), ('умивальник', 0.5056540369987488), ('під

[('лечение', 0.8136386871337891), ('четырех', 0.8058596849441528), ('восемь', 0.8057748079299927), ('первенство', 0.8018766045570374), ('моей', 0.8004988431930542), ('детской', 0.7960490584373474), ('четвертом', 0.7947453260421753), ('шесть', 0.7944707274436951), ('операции', 0.7943021059036255), ('американской', 0.7940250635147095)]
Category: Заборгованість-з-виплати-пенсій.txt
[('прибыль', 0.5877031087875366), ('банк', 0.5867656469345093), ('реконструкцию', 0.5859189629554749), ('банке', 0.5772806406021118), ('сумма', 0.5750601291656494), ('аппарат', 0.5727213621139526), ('вдвое', 0.5713111162185669), ('бизнесмен', 0.5694906711578369), ('шесть', 0.56740403175354), ('авиалинии', 0.5671948790550232)]
Category: Забруднення-повітря-від-невідомих-джерел.txt
[('воды', 0.6981723308563232), ('кладбища', 0.6694822311401367), ('ленина', 0.6665803790092468), ('площади', 0.6652618646621704), ('домов', 0.6651495695114136), ('пещер', 0.6637709140777588), ('протестующих', 0.6608668565750122), ('ули

[('кондиціонера', 0.4731677174568176), ('жек', 0.46171140670776367), ('ліфт', 0.46164363622665405), ('туалет', 0.44469231367111206), ('телефонувати', 0.4417645335197449), ('віконця', 0.4411490559577942), ('кондиціонер', 0.44070684909820557), ('туалету', 0.4405750632286072), ('холодильник', 0.4331270158290863), ('ліжка', 0.4297635853290558)]
Category: Не-працює-пасажирський-ліфт.txt
[('3', 0.612985372543335), ('7', 0.6092515587806702), ('2', 0.6091914176940918), ('6', 0.6064190864562988), ('8', 0.6041659116744995), ('восемь', 0.602942943572998), ('услуги', 0.598538339138031), ('1', 0.593137264251709), ('5', 0.5920297503471375), ('4', 0.5911935567855835)]
Category: Недостатній-тиск-ГВП.txt
[('водопостачання', 0.5088596343994141), ('опалення', 0.5038186311721802), ('водопроводі', 0.5025632381439209), ('теплоносія', 0.49973636865615845), ('тепло', 0.49239370226860046), ('обігрів', 0.4914620518684387), ('тец-6', 0.4901392459869385), ('водозабір', 0.48839402198791504), ('каналізацію', 0.4849

...

[('приміщення', 0.5257163047790527), ('автостоянки', 0.5006952881813049), ('будівлі', 0.49588096141815186), ('гуртожитку', 0.4830384850502014), ('приміщеннях', 0.4808194041252136), ('будинку', 0.47904419898986816), ('адмінбудинку', 0.47414860129356384), ('кафе', 0.47404205799102783), ('помещения', 0.47397541999816895), ('приміщень', 0.4731970429420471)]
Category: Перевірка-дозвільної-документації--демонтаж-окремо-розташованих-гаражів.txt
[('веранді', 0.658745288848877), ('підвіконні', 0.6425780057907104), ('підвіконня', 0.6347206234931946), ('балконі', 0.6318095922470093), ('веранди', 0.5976603031158447), ('сараї', 0.5974125266075134), ('лавках', 0.5943784713745117), ('грубки', 0.5927984118461609), ('горищі', 0.589855432510376), ('хлів', 0.5879753232002258)]
Category: Перевірка-дозвільної-документації--демонтаж-торгових-палаток--пивних-наметів.txt
[('намет', 0.6463233232498169), ('кіоск', 0.637466549873352), ('палатки', 0.5602568984031677), ('лавочки', 0.5537140965461731), ('лотки', 0.

[('обещают', 0.8697035312652588), ('девушки', 0.8695540428161621), ('предложили', 0.8684965372085571), ('увидев', 0.8682301640510559), ('уйти', 0.8672879934310913), ('моей', 0.8660528063774109), ('нечто', 0.8660440444946289), ('медленно', 0.8656291365623474), ('конкретные', 0.8649396896362305), ('умереть', 0.8648123741149902)]
Category: Порушення-громадського-порядку-безпритульними-людьми.txt
[('словно', 0.8401201963424683), ('преимущественно', 0.8362420797348022), ('девушку', 0.8361560106277466), ('спит', 0.8335753679275513), ('деревья', 0.8321963548660278), ('ребят', 0.8318508267402649), ('искали', 0.8311629295349121), ('девушки', 0.8308429718017578), ('массы', 0.8306678533554077), ('увидев', 0.8279339671134949)]
Category: Порушення-посадової-інструкції-водієм-автобусу.txt
[('пешком', 0.7074652314186096), ('водитель', 0.6873271465301514), ('восемь', 0.6732947826385498), ('ехать', 0.6690701842308044), ('словно', 0.6671222448348999), ('молча', 0.6650832295417786), ('туда', 0.6629942655

[('2', 0.7913042306900024), ('3', 0.7886085510253906), ('1', 0.7786535620689392), ('4', 0.7539829015731812), ('6', 0.7525811195373535), ('7', 0.7473628520965576), ('8', 0.7359597682952881), ('5', 0.7295332551002502), ('9', 0.7262169122695923), ('15', 0.7250908613204956)]
Category: Проведення-модернізації-заміни-ліфту.txt
[('проводять', 0.4292629361152649), ('проводитимуть', 0.40838223695755005), ('проводимо', 0.39249974489212036), ('експлуатацію', 0.3879297077655792), ('тепломереж', 0.377359002828598), ('проводили', 0.3749461770057678), ('проводяться', 0.37465953826904297), ('проводитимуться', 0.3707703649997711), ('проведено', 0.35941600799560547), ('електроустановок', 0.35717788338661194)]
Category: Продаж-спиртних-слабоалкогольних-напоїв-та-тютюнових-виробів-особам-молодше---р-.txt
[('автостоянки', 0.46930035948753357), ('паркування', 0.44106149673461914), ('землекористувачів', 0.4350265860557556), ('мафів', 0.4331378936767578), ('землевідведення', 0.43110114336013794), ('пущі-водиц

[('приміщень', 0.5175008773803711), ('документів', 0.5125552415847778), ('погоджень', 0.4968644380569458), ('довідок', 0.495021790266037), ('відповідних', 0.48917919397354126), ('погодження', 0.48052045702934265), ('землевідведення', 0.4800758957862854), ('інструкцій', 0.47696149349212646), ('перевірки', 0.4721059203147888), ('закладів', 0.4659942090511322)]
Category: Робота-оперативної-служби------.txt
[('вашей', 0.8703181743621826), ('моей', 0.8698508739471436), ('наконец', 0.856308102607727), ('дворца', 0.8522160053253174), ('вскоре', 0.8506765365600586), ('пытаясь', 0.8503267168998718), ('обвинение', 0.8503260612487793), ('операции', 0.8502520322799683), ('милицию', 0.8499717116355896), ('кабинет', 0.8494375944137573)]
Category: Робота-світлофора.txt
[('перехрестя', 0.5716425180435181), ('набережно-хрещатицької', 0.5568650960922241), ('світлофора', 0.539433479309082), ('набережно-', 0.5384761095046997), ('світлофор', 0.5306098461151123), ('личаківську', 0.5258426666259766), ('набер

[('документы', 0.4956126809120178), ('старой', 0.4870266616344452), ('возле', 0.48613664507865906), ('счета', 0.4855623245239258), ('моей', 0.4838012754917145), ('надпись', 0.48377376794815063), ('здание', 0.4809971749782562), ('обвинения', 0.47622257471084595), ('наличие', 0.4761604368686676), ('ленина', 0.47514742612838745)]
Category: Утримання-домашніх-тварин.txt
[('комнату', 0.7953535318374634), ('вашей', 0.7940720319747925), ('парень', 0.7903386354446411), ('словно', 0.7878551483154297), ('будто', 0.7860885262489319), ('молчать', 0.7850061655044556), ('увидев', 0.7837495803833008), ('помните', 0.7832763195037842), ('дверь', 0.7815505266189575), ('девушку', 0.7805368304252625)]
Category: Утримання-номерних-знаків-і-табличок-з-назвами-вулиць-на-будинках--і-спорудах.txt
[('бортничі', 0.440458208322525), ('будинку', 0.43274664878845215), ('волоській', 0.4295019507408142), ('личаківську', 0.4166424870491028), ('жилянській', 0.4165271520614624), ('пасічної', 0.4134921431541443), ('район

In [64]:
vec = text2vec('один текст')
print(vec[:5])
vec = text2vec('звертаюсь до вас')
print(vec[:5])

[ 0.0247874   0.00025589  0.04356825 -0.05381794 -0.02316727]
[-0.10483106  0.02657524 -0.11398544 -0.04298301 -0.12793895]
